In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import glob

In [ ]:
# 컬럼명 정의
column_names = ['timestamp', 'v_raw', 'c_raw', 'voltage', 'current', 'label']

# 정상 데이터
normal_dir = '../realtime/normal/*/'
normal_files = glob.glob(os.path.join(normal_dir, '*.csv'))
normal_dfs = [pd.read_csv(file, names=column_names, header=None) for file in normal_files]
normal_data = pd.concat(normal_dfs, ignore_index=True)

# 아크 데이터
arc_dir = '../realtime/arc/*/'
arc_files = glob.glob(os.path.join(arc_dir, '*.csv'))
arc_dfs = [pd.read_csv(file, names=column_names, header=None) for file in arc_files]
arc_data = pd.concat(arc_dfs, ignore_index=True)

# 전체 병합 및 셔플
train_df = pd.concat([normal_data, arc_data], ignore_index=True)
train_df = train_df.sample(frac=1).reset_index(drop=True)

In [4]:
# 파생 피처
def add_features(df):
    df['voltage_diff'] = df['voltage'].diff().fillna(0).abs()
    df['current_diff'] = df['current'].diff().fillna(0).abs()
    df['voltage_ma'] = df['voltage'].rolling(5).mean().bfill()
    df['current_ma'] = df['current'].rolling(5).mean().bfill()
    df['power'] = df['voltage'] * df['current']
    df['power_diff'] = df['power'].diff().fillna(0).abs()
    return df

df = add_features(train_df).dropna()

In [5]:
features = ['voltage', 'current', 'voltage_diff', 'current_diff',
            'voltage_ma', 'current_ma', 'power', 'power_diff']

X_raw = df[features].values
y_raw = df['label'].values.astype(int)

# 정규화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_raw)

In [6]:
# 시퀀스 생성
def create_sequences(data, labels, seq_len=8):
    X, y = [], []
    for i in range(len(data) - seq_len):
        X.append(data[i:i+seq_len])
        y.append(labels[i+seq_len - 1])
    return np.array(X), np.array(y)

seq_len = 8
X_seq, y_seq = create_sequences(X_scaled, y_raw, seq_len)
X_seq = X_seq.reshape((X_seq.shape[0], seq_len, len(features)))

In [8]:
# 학습/검증 분할
X_train, X_val, y_train, y_val = train_test_split(X_seq, y_seq, test_size=0.2, stratify=y_seq, random_state=42)

In [9]:
inputs = Input(shape=(seq_len, len(features)))
x = Conv1D(64, 3, activation='relu', padding='same')(inputs)
x = MaxPooling1D(2)(x)
x = Dropout(0.3)(x)
x = Conv1D(128, 3, activation='relu', padding='same')(x)
x = MaxPooling1D(2)(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)
outputs = Dense(1, activation='sigmoid')(x)

model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 학습
early_stop = EarlyStopping(patience=5, restore_best_weights=True)
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=64, callbacks=[early_stop])

2025-05-06 14:47:17.097532: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-05-06 14:47:17.097707: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-05-06 14:47:17.097716: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-05-06 14:47:17.097916: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-05-06 14:47:17.097933: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/50


2025-05-06 14:47:17.899256: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


342/342 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - accuracy: 0.8093 - loss: 0.5079 - val_accuracy: 0.8205 - val_loss: 0.4531
Epoch 2/50
342/342 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.8185 - loss: 0.4678 - val_accuracy: 0.8371 - val_loss: 0.4347
Epoch 3/50
342/342 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.8314 - loss: 0.4451 - val_accuracy: 0.8412 - val_loss: 0.4214
Epoch 4/50
342/342 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.8401 - loss: 0.4240 - val_accuracy: 0.8437 - val_loss: 0.4123
Epoch 5/50
342/342 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.8438 - loss: 0.4169 - val_accuracy: 0.8452 - val_loss: 0.4017
Epoch 6/50
342/342 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.8468 - loss: 0.4043 - val_accuracy: 0.8489 - val_loss: 0.4032
Epoch 7/50
342/342 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.8449 - loss: 0.4058 - val_accuracy: 0.8472 - val_loss: 0.3968
Epoch 8/50
342/342 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.8465 - loss: 0.4058 - val_accuracy: 0.849

In [10]:
# 모델 및 스케일러 저장
model.save('./model/cnn1d_model.h5')
joblib.dump(scaler, './model/scaler_cnn.joblib')

['./model/scaler_cnn.joblib']

In [11]:
import tensorflow as tf

model = tf.keras.models.load_model('./model/cnn1d_model.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)

# 핵심 설정: 구버전 연산자만 사용하도록 제한
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]  

# (선택) float16 경량화도 가능
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.target_spec.supported_types = [tf.float16]

tflite_model = converter.convert()

with open("./model/cnn1d_model_compatible.tflite", "wb") as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: /var/folders/k8/zsc42w0n02g8ywyn_nyjq8140000gn/T/tmp961xp7_x/assets


INFO:tensorflow:Assets written to: /var/folders/k8/zsc42w0n02g8ywyn_nyjq8140000gn/T/tmp961xp7_x/assets


: 